Neural Networks with TensorFlow (Follow Along)
===

We are going to learn how to build a TensorFlow implementation of an Artificial Neural Network. More especifically, we will learn how to classify a set of images using a Convolutional Neural Network. We will cover the following sections:  

1. A very simple MNIST Classifier
2. Visualizing our Results
3. tf.layers

**Note:** This document is part of the "Neural Networks with TensorFlow" workshop, which is meant to accompany these [slides](https://docs.google.com/presentation/d/185r0vPc4gzE_-4dOsjy0JmO8qk2M7BBm2Uhqg5tAF7E/edit?usp=sharing). The main source for this notebook is https://www.tensorflow.org/get_started/mnist/beginners. For more information please visit [AI for Everyone!](http://andresvourakis.com/ai-for-everyone)

A very simple MNIST Classifier
---

In this section we will create a **softmax regression** function that is a model for recognizing MNIST digits, based on looking at every pixel in the image.

![](https://www.tensorflow.org/images/MNIST.png)

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports

# Import dataset

# Create Model

# Define Loss and Optimizer

# Train

# Test Trained Model

Visualizing our results
---

In [ ]:
def visualize_weights():
    for i in range(10):
        plt.subplot(2, 5, i+1)
        weight = sess.run(W)[:,i]
        plt.title(i)
        plt.imshow(weight.reshape([28,28]), cmap=plt.get_cmap('seismic'))
        frame1 = plt.gca()
        frame1.axes.get_xaxis().set_visible(False)
        frame1.axes.get_yaxis().set_visible(False) 
    plt.show()

def compare_prediction(num):
    # Load test example
    x_train = mnist.test.images[num,:].reshape(1,784)
    y_train = mnist.test.labels[num,:]
    
    # Get label as integer
    label = y_train.argmax()
    
    # Get prediction as integer
    prediction = sess.run(y_, feed_dict={x: x_train}).argmax() 
    
    plt.title('Prediction: %d Label: %d' % (prediction, label))
    plt.imshow(x_train.reshape([28,28]), cmap=plt.get_cmap('gray_r'))
    plt.show()
        

In [ ]:
# Visualize the weights


In [ ]:
# Compare prediction with label


Let's dive a little deeper (tf.layers)
---


What if we wanted to build an Artificial Neural Network from scratch? Well, TensorFlow provides us with **tf.layers**, a set of high-level neural networks layers.

One of the most popular techniques used in improving the accuracy of image classification is Convolutional Neural Networks, let's see how one would go about using **tf.layers** to do that.

1. **tf.layers.conv2d()**  
Constructs a two-dimensional convolutional layer. Takes number of filters, filter kernel size, padding, and activation function as arguments.

2. **tf.layers.max_pooling2d()**  
Constructs a two-dimensional pooling layer using the max-pooling algorithm. Takes pooling filter size and stride as arguments.

3. **tf.layers.dense()**  
Constructs a dense layer. Takes number of neurons and activation function as arguments.

Let's review this Convolutional Neural Network model
---

![](https://ujwlkarn.files.wordpress.com/2016/08/screen-shot-2016-08-07-at-4-59-29-pm.png?w=748)

In [ ]:
def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1]) # [batch_size, image_width, image_height, channels]

  # Convolutional Layer #1
  # Extract features from input image    
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu) # Introduce Non-linearity

  #Pooling Layer #1
  # Reduce Dimensionality and extract most important features
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
  # Extract more features and reduce dimensionality    
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Dense Layer
  # Look at all features extracted and perform Classification   
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
  loss = tf.losses.softmax_cross_entropy(
      onehot_labels=onehot_labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

For more infomation see https://www.tensorflow.org/tutorials/layers